In [1]:
import pvporcupine
import sounddevice as sd

# Configuración de Porcupine con access_key
porcupine = pvporcupine.create(
    access_key="qEcRoI9pyPji4cEIXkMNz4+G34KlNQMO5mouLLJzGsvb5PF5fuaKlg==",  # Reemplaza con tu clave de acceso
    keyword_paths=["resources/wakeword.ppn"],
    model_path="resources/porcupine_params_es.pv" 
)

def wake_word_detected():
    print("Wake word detectado. Escuchando tu comando...")
import numpy as np

def audio_callback(indata, frames, time, status):
    if len(indata) != porcupine.frame_length:
        print(f"Ajustando frame length. Original: {len(indata)}, esperado: {porcupine.frame_length}")
        # Interpolamos para ajustar el tamaño del frame
        factor = porcupine.frame_length / len(indata)
        indata = np.interp(
            np.arange(0, len(indata) * factor) / factor,  # Nuevas posiciones
            np.arange(len(indata)),  # Posiciones originales
            indata[:, 0]  # Solo el primer canal
        ).astype("int16")

    pcm = indata  # Ya está ajustado
    keyword_index = porcupine.process(pcm)
    if keyword_index >= 0:
        wake_word_detected()



In [2]:
import whisper
import sounddevice as sd
import numpy as np

# Cargar el modelo de Whisper
model = whisper.load_model("base")

def transcribe_audio():
    print("Escuchando comando...")
    duration = 5  # Duración en segundos
    audio = sd.rec(int(duration * 16000), samplerate=16000, channels=1, dtype='float32')
    sd.wait()  # Esperar a que termine la grabación
    
    # Guardar el audio en un archivo temporal
    np.save("audio_temp.npy", audio)
    print("Transcribiendo...")
    
    # Usar Whisper para convertir audio a texto
    result = model.transcribe("audio_temp.npy", language="es")
    return result['text']


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

In [3]:
def process_command(command):
    print(f"Comando recibido: {command}")
    genre = None
    runtime = None

    # Ejemplo de procesamiento básico
    if "amor" in command:
        genre = "romance"
    if "menos de 1:30" in command or "corta" in command:
        runtime = 90

    return {
        "genre": genre,
        "runtime": runtime
    }


In [4]:
import requests

# URL del servidor Flask
BASE_URL = "http://127.0.0.1:5000"

# Obtener la lista de seguimiento (después de autenticarse)
response = requests.get(f"{BASE_URL}/get_watchlist")
print(response.json())


{'results': [{'adult': False, 'backdrop_path': '/8mjYwWT50GkRrrRdyHzJorfEfcl.jpg', 'genre_ids': [28, 12, 18], 'id': 558449, 'original_language': 'en', 'original_title': 'Gladiator II', 'overview': 'Years after witnessing the death of the revered hero Maximus at the hands of his uncle, Lucius is forced to enter the Colosseum after his home is conquered by the tyrannical Emperors who now lead Rome with an iron fist. With rage in his heart and the future of the Empire at stake, Lucius must look to his past to find strength and honor to return the glory of Rome to its people.', 'popularity': 2557.256, 'poster_path': '/2cxhvwyEwRlysAmRH4iodkvo0z5.jpg', 'release_date': '2024-11-13', 'title': 'Gladiator II', 'video': False, 'vote_average': 6.836, 'vote_count': 341}, {'adult': False, 'backdrop_path': '/18TSJF1WLA4CkymvVUcKDBwUJ9F.jpg', 'genre_ids': [27, 53, 9648], 'id': 1034541, 'original_language': 'en', 'original_title': 'Terrifier 3', 'overview': "Five years after surviving Art the Clown's 

In [5]:
def detect_wake_word():
    def audio_callback(indata, frames, time, status):
        if status:
            print(f"Estado del stream: {status}")
        
        # Verifica si el tamaño del frame es correcto
        if len(indata) == porcupine.frame_length:
            pcm = indata[:, 0]  # Procesa solo el canal mono
            keyword_index = porcupine.process(pcm)
            if keyword_index >= 0:
                wake_word_detected()
        else:
            print(f"Frame length inválido: esperado {porcupine.frame_length}, recibido {len(indata)}")

    # Configurar el flujo de audio con blocksize
    with sd.InputStream(
        callback=audio_callback,
        channels=1,  # Mono
        samplerate=porcupine.sample_rate,  # 16000 Hz
        blocksize=porcupine.frame_length,  # Asegurar el tamaño del bloque
        dtype="int16",  # Formato esperado por Porcupine
    ):
        print("Escuchando activación...")
        while True:
            pass



In [6]:
import pvporcupine
import sounddevice as sd
import numpy as np
import time
from scipy.io.wavfile import write
import os

DEVICE_INDEX = 5  # Ajustar al índice correcto según tu micrófono
running = True  # Bandera para controlar el bucle principal

# Configuración de Porcupine
try:
    porcupine = pvporcupine.create(
        access_key="qEcRoI9pyPji4cEIXkMNz4+G34KlNQMO5mouLLJzGsvb5PF5fuaKlg==",
        keyword_paths=["resources/wakeword.ppn"],
        model_path="resources/porcupine_params_es.pv"
    )
except Exception as e:
    print(f"Error al inicializar Porcupine: {e}")
    exit(1)

def wake_word_detected():
    global running
    print("Wake word detectado. Preparando para escuchar el comando...")
    grabar_comando()
    running = False  # Detener el bucle principal

def grabar_comando():
    print("Grabando audio por 5 segundos...")
    try:
        duration = 5
        audio = sd.rec(int(duration * 16000), samplerate=16000, channels=1, dtype='int16', device=DEVICE_INDEX)
        sd.wait()
        print("Grabación completada.")
        
        # Guardar el archivo en formato estándar para Whisper
        temp_audio_path = "temp_audio.wav"
        write(temp_audio_path, 16000, audio)
        print(f"Audio guardado en {os.path.abspath(temp_audio_path)}")
    except Exception as e:
        print(f"Error al grabar audio: {e}")

def audio_callback(indata, frames, time, status):
    if status:
        print(f"Estado del stream: {status}")
    try:
        pcm = indata[:, 0]
        keyword_index = porcupine.process(pcm)
        if keyword_index >= 0:
            wake_word_detected()
    except Exception as e:
        print(f"Error en el callback: {e}")

def detect_wake_word():
    global running
    print("Iniciando el sistema de detección de wake word...")
    try:
        with sd.InputStream(
            device=DEVICE_INDEX,
            callback=audio_callback,
            channels=1,
            samplerate=porcupine.sample_rate,
            blocksize=porcupine.frame_length,
            dtype="int16",
            latency="low"
        ) as stream:
            print("Escuchando activación...")
            while running:
                time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nInterrupción detectada. Finalizando el programa...")
    except Exception as e:
        print(f"Error en el sistema de detección: {e}")
    finally:
        porcupine.delete()
        print("Sistema de detección finalizado.")

if __name__ == "__main__":
    detect_wake_word()


Iniciando el sistema de detección de wake word...
Escuchando activación...
Wake word detectado. Preparando para escuchar el comando...
Grabando audio por 5 segundos...
Grabación completada.
Audio guardado en c:\Users\alexc\OneDrive - Universidad Politécnica de Madrid\Escritorio\CDIA\Proyectos\Film_recomendations\temp_audio.wav
Sistema de detección finalizado.


In [7]:
import whisper
import os

os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"
# Cargar el modelo de Whisper
print("Cargando modelo de Whisper...")
model = whisper.load_model("base")

def transcribe_audio(filename):
    """Transcribir audio usando Whisper"""
    if os.path.exists(filename):
        print(f"Archivo encontrado: {os.path.abspath(filename)}")
        print("Procesando transcripción...")
        try:
            # Asegúrate de que el archivo está accesible y tiene el formato correcto
            result = model.transcribe(filename, language="es")
            print(f"Transcripción: {result['text']}")
        except Exception as e:
            print(f"Error al transcribir el audio: {e}")
    else:
        print(f"Error: El archivo {filename} no existe o no es accesible.")

# Ruta del archivo generado en la parte anterior
wav_filename = "temp_audio.wav"
transcribe_audio(wav_filename)


Cargando modelo de Whisper...


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Archivo encontrado: c:\Users\alexc\OneDrive - Universidad Politécnica de Madrid\Escritorio\CDIA\Proyectos\Film_recomendations\temp_audio.wav
Procesando transcripción...


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción: 


In [8]:
import requests

# URL del servidor Flask
BASE_URL = "http://127.0.0.1:5000"

# Endpoint para obtener la watchlist
endpoint = "/get_watchlist"

# Hacer la solicitud GET
response = requests.get(f"{BASE_URL}{endpoint}")

# Verificar el resultado
if response.status_code == 200:
    watchlist = response.json()
    print("Películas en la Watchlist:")
    for movie in watchlist["results"]:
        print(f"- {movie['title']} ({movie['release_date']}): {movie['overview']}")
else:
    print(f"Error {response.status_code}: {response.text}")


Películas en la Watchlist:
- Gladiator II (2024-11-13): Years after witnessing the death of the revered hero Maximus at the hands of his uncle, Lucius is forced to enter the Colosseum after his home is conquered by the tyrannical Emperors who now lead Rome with an iron fist. With rage in his heart and the future of the Empire at stake, Lucius must look to his past to find strength and honor to return the glory of Rome to its people.
- Terrifier 3 (2024-10-09): Five years after surviving Art the Clown's Halloween massacre, Sienna and Jonathan are still struggling to rebuild their shattered lives. As the holiday season approaches, they try to embrace the Christmas spirit and leave the horrors of the past behind. But just when they think they're safe, Art returns, determined to turn their holiday cheer into a new nightmare. The festive season quickly unravels as Art unleashes his twisted brand of terror, proving that no holiday is safe.
- Dilwale Dulhania Le Jayenge (1995-10-20): Raj is a

In [9]:
def classify_genre(transcription):
    """
    Clasifica todos los géneros mencionados en la transcripción a sus IDs correspondientes en TMDb.
    """
    genre_dict = {
        "acción": 28,
        "aventura": 12,
        "animación": 16,
        "comedia": 35,
        "crimen": 80,
        "documental": 99,
        "drama": 18,
        "familia": 10751,
        "fantasía": 14,
        "historia": 36,
        "terror": 27,
        "música": 10402,
        "misterio": 9648,
        "romance": 10749,
        "ciencia ficción": 878,
        "película de tv": 10770,
        "thriller": 53,
        "bélico": 10752,
        "western": 37
    }

    transcription = transcription.lower()

    # Buscar todos los géneros mencionados
    genres_found = [
        genre_id for genre_name, genre_id in genre_dict.items()
        if genre_name in transcription
    ]

    return genres_found if genres_found else None


In [10]:
import spacy

# Asegúrate de tener este modelo
nlp = spacy.load("es_core_news_md")
def process_transcription_to_params(transcription):
    """
    Transforma el texto de la transcripción en parámetros para la solicitud.
    """
    transcription = transcription.lower()
    
    # Inicializar parámetros
    params = {
        "vote_count": None,
        "vote_average": None,
        "after_year": None,
        "before_year": None,
        "adult": None,
        "language": None,
        "genre": None
    }
    
    # Clasificar el género
    params["genre"] = classify_genre(transcription)
    
    # Identificar si es para adultos o familiar
    if "adulto" in transcription:
        params["adult"] = True
    elif "familiar" in transcription:
        params["adult"] = False
    
    # Procesar números (años y puntuaciones)
    nlp_doc = nlp(transcription)
    for token in nlp_doc:
        if token.like_num:
            number = float(token.text)
            
            # Identificar votación promedio
            if any(keyword in transcription for keyword in ["más de", "mayor a", "superior a", "por encima de"]):
                params["vote_average"] = number
            
            # Identificar años
            if len(token.text) == 4:  # Probable año
                if "antes de" in transcription:
                    params["before_year"] = int(number)
                elif "después de" in transcription or "posterior a" in transcription:
                    params["after_year"] = int(number)

    # Identificar idioma
    if "español" in transcription:
        params["language"] = "es"
    elif "inglés" in transcription:
        params["language"] = "en"
    elif "francés" in transcription:
        params["language"] = "fr"
    
    return params


In [11]:
# Texto de entrada
transcription = "Quiero una película de ciencia ficción o acción que sea de después de 2020 y con una nota superior a 5."

# Transformar el texto en parámetros
params = process_transcription_to_params(transcription)
print(params)


{'vote_count': None, 'vote_average': 5.0, 'after_year': 2020, 'before_year': None, 'adult': None, 'language': None, 'genre': [28, 878]}


In [12]:
import requests

params = params
BASE_URL = "http://127.0.0.1:5000"
response = requests.get(f"{BASE_URL}/get_watchlist", params=params)
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code}")


{'results': [{'adult': False, 'backdrop_path': '/8mjYwWT50GkRrrRdyHzJorfEfcl.jpg', 'genre_ids': [28, 12, 18], 'id': 558449, 'original_language': 'en', 'original_title': 'Gladiator II', 'overview': 'Years after witnessing the death of the revered hero Maximus at the hands of his uncle, Lucius is forced to enter the Colosseum after his home is conquered by the tyrannical Emperors who now lead Rome with an iron fist. With rage in his heart and the future of the Empire at stake, Lucius must look to his past to find strength and honor to return the glory of Rome to its people.', 'popularity': 2557.256, 'poster_path': '/2cxhvwyEwRlysAmRH4iodkvo0z5.jpg', 'release_date': '2024-11-13', 'title': 'Gladiator II', 'video': False, 'vote_average': 6.836, 'vote_count': 341}, {'adult': False, 'backdrop_path': '/18TSJF1WLA4CkymvVUcKDBwUJ9F.jpg', 'genre_ids': [27, 53, 9648], 'id': 1034541, 'original_language': 'en', 'original_title': 'Terrifier 3', 'overview': "Five years after surviving Art the Clown's 

In [13]:
!pip install pyttsx3

In [23]:
!pip install soundfile transformers


In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hola, mi perro es mono.", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)


In [4]:
from datasets import load_from_disk
import os
# Obtener la ruta actual
current_path = os.getcwd()
print(f"Ruta actual: {current_path}")

# Especificar el nombre del directorio para guardar/cargar el dataset
dataset_path = os.path.join(current_path, "tokenized_datasets")

# Cargar el dataset tokenizado
tokenized_datasets = load_from_disk(dataset_path)
print(tokenized_datasets)


Ruta actual: c:\Proyectos\Film_recomendations


Loading dataset from disk:   0%|          | 0/76 [00:00<?, ?it/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 230467
})


In [5]:
from datasets import Dataset

# Toma solo una fracción del dataset
reduction_factor = 0.005  # Usa solo el 10% del dataset
small_dataset = tokenized_datasets.select(range(int(len(tokenized_datasets) * reduction_factor)))


In [6]:
from datasets import Dataset
import torch
import torch.nn.functional as F

def split_dataset(dataset, num_splits=2):
    """
    Divide un dataset en partes iguales.
    """
    split_size = len(dataset) // num_splits
    splits = [dataset[i * split_size: (i + 1) * split_size] for i in range(num_splits)]
    if len(dataset) % num_splits != 0:  # Si queda algo al final, lo añadimos a la última parte
        splits[-1].extend(dataset[num_splits * split_size:])
    return splits

def preprocess_and_align(dataset, pad_token_id=0, max_label_length=500):
    """
    Preprocesa y alinea un dataset completo moviendo los tensores necesarios a CUDA.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    aligned_dataset = []

    print("Procesando el dataset en GPU (si está disponible)...")

    for example in dataset:
        # Convertir listas a tensores y mover a GPU
        input_ids_tensor = torch.tensor(example["input_ids"], dtype=torch.long).to(device)
        labels_tensor = torch.tensor(example["labels"], dtype=torch.float32).to(device)

        # Ajustar `labels` para que tengan exactamente max_label_length
        if labels_tensor.shape[0] > max_label_length:
            aligned_labels = labels_tensor[:max_label_length]
        else:
            aligned_labels = F.pad(labels_tensor, (0, max_label_length - labels_tensor.shape[0]))

        # Ajustar `input_ids`
        padded_input_ids = F.pad(input_ids_tensor, (0, max_label_length - len(input_ids_tensor)), value=pad_token_id)

        # Crear un nuevo ejemplo alineado
        aligned_example = {
            "input_ids": padded_input_ids[:max_label_length],
            "attention_mask": torch.ones_like(padded_input_ids[:max_label_length]),
            "labels": aligned_labels
        }
        aligned_dataset.append({k: v.cpu() for k, v in aligned_example.items()})  # Pasar a CPU para liberar memoria

    print("Preprocesamiento y alineamiento completados.")
    return aligned_dataset

# Dividir el dataset en dos mitades
if isinstance(small_dataset, Dataset):
    small_dataset = [example for example in small_dataset if len(example["labels"]) > 0]


dataset_splits = split_dataset(small_dataset, num_splits=2)

# Procesar la primera mitad
print("Procesando la primera mitad del dataset...")
aligned_dataset_part1 = preprocess_and_align(dataset_splits[0], pad_token_id=0, max_label_length=500)

# Entrenar con la primera mitad del dataset alineado
print(f"Tamaño del dataset alineado (parte 1): {len(aligned_dataset_part1)}")

# Procesar la segunda mitad (si es necesario más adelante)
# print("Procesando la segunda mitad del dataset (opcional)...")
# aligned_dataset_part2 = preprocess_and_align(dataset_splits[1], pad_token_id=0, max_label_length=500)

# Validar algunos ejemplos de la primera mitad
print("Validando ejemplos alineados de la primera mitad:")
for idx, example in enumerate(aligned_dataset_part1[:5]):  # Muestra los primeros 5 ejemplos
    print(f"Ejemplo {idx}:")
    print(f"input_ids shape: {example['input_ids'].shape}")
    print(f"labels shape: {example['labels'].shape}")
    print(f"attention_mask shape: {example['attention_mask'].shape}")


Procesando la primera mitad del dataset...
Procesando el dataset en GPU (si está disponible)...
Preprocesamiento y alineamiento completados.
Tamaño del dataset alineado (parte 1): 576
Validando ejemplos alineados de la primera mitad:
Ejemplo 0:
input_ids shape: torch.Size([500])
labels shape: torch.Size([500, 80])
attention_mask shape: torch.Size([500])
Ejemplo 1:
input_ids shape: torch.Size([500])
labels shape: torch.Size([500, 80])
attention_mask shape: torch.Size([500])
Ejemplo 2:
input_ids shape: torch.Size([500])
labels shape: torch.Size([500, 80])
attention_mask shape: torch.Size([500])
Ejemplo 3:
input_ids shape: torch.Size([500])
labels shape: torch.Size([500, 80])
attention_mask shape: torch.Size([500])
Ejemplo 4:
input_ids shape: torch.Size([500])
labels shape: torch.Size([500, 80])
attention_mask shape: torch.Size([500])


In [7]:
import torch
torch.cuda.empty_cache()  # Limpia la memoria no utilizada
print(torch.cuda.memory_summary())  # Verifica el uso actual de memoria



|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   | 338944 B   |  95616 KiB |  95616 KiB |
|       from large pool |      0 B   |      0 B   |      0 KiB |      0 KiB |
|       from small pool |      0 B   | 338944 B   |  95616 KiB |  95616 KiB |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   | 338944 B   |  95616 KiB |  95616 KiB |
|       from large pool |      0 B   |      0 B   |      0 KiB |

In [8]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, Trainer, TrainingArguments


processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_total_limit=2,
    fp16=True,  # Habilitar precisión mixta
)

 
# Configurar el Trainer sin eval_dataset ni compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=aligned_dataset_part1,  # Solo el conjunto de entrenamiento
    data_collator=None,  # Ya alineaste los datos, así que no necesitas data_collator
)

# Entrenar el modelo
trainer.train()

  0%|          | 0/864 [00:00<?, ?it/s]

c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 5e-05, 'epoch': 1.74}
{'train_runtime': 196.9177, 'train_samples_per_second': 8.775, 'train_steps_per_second': 4.388, 'train_loss': 0.0, 'epoch': 3.0}


TrainOutput(global_step=864, training_loss=0.0, metrics={'train_runtime': 196.9177, 'train_samples_per_second': 8.775, 'train_steps_per_second': 4.388, 'total_flos': 748305195264000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [10]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import Dataset
import torchaudio

# Cargar el modelo y el procesador desde el checkpoint
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")  # Último checkpoint
model = SpeechT5ForTextToSpeech.from_pretrained("./results/checkpoint-864")
hifi_gan = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")  # Vocoder para mejorar el audio


In [6]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf

# Cargar el procesador, modelo y vocoder
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("./results/checkpoint-864")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Texto de prueba
inputs = processor(text="Hola, mi perro es mono.", return_tensors="pt")

# Cargar embeddings del hablante
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Generar espectrograma con el modelo
with torch.no_grad():
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings=speaker_embeddings, vocoder=vocoder)

# Guardar el archivo de audio
sf.write("speech.wav", speech.numpy(), samplerate=16000)  # 16kHz es la tasa de muestreo estándar
print("Audio generado y guardado en 'speech.wav'")


Audio generado y guardado en 'speech.wav'


In [13]:
from datasets import load_dataset

# Cargar el conjunto de datos de Common Voice para el idioma español
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="train")

# Inspeccionar el conjunto de datos
print(dataset)
print(dataset[0])  # Primer ejemplo


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 230467
})
{'client_id': '34719bb7c7344da7733b85c9d7215d24326093f1a2cd3a445bdc6dfe9ec4a8c9fe9729a73f6c29764545276bff81ffa65d3944f6da7a3ee3c06d0eb124fac797', 'path': 'C:\\Users\\alexc\\.cache\\huggingface\\datasets\\downloads\\extracted\\2b79cc6219206153d43fce3a34af5d3200bd34fe6f7778eb294ec55a3a7d7854\\es_train_0/common_voice_es_18338585.mp3', 'audio': {'path': 'C:\\Users\\alexc\\.cache\\huggingface\\datasets\\downloads\\extracted\\2b79cc6219206153d43fce3a34af5d3200bd34fe6f7778eb294ec55a3a7d7854\\es_train_0/common_voice_es_18338585.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.25855126e-06, -3.52389725e-06, -3.05285812e-06]), 'sampling_rate': 48000}, 'sentence': '¿ Qué tal a tres de cinco ?', 'up_votes': 2, 'down_votes': 1, 'age': '', 'gender': '', 'accent': '', 'locale': 'es', 'segment': ''}


In [15]:
import torchaudio
import torch

def process_audio(batch):
    # Convertir el array de numpy a tensor con tipo float32
    waveform = torch.tensor(batch["audio"]["array"], dtype=torch.float32)
    original_rate = batch["audio"]["sampling_rate"]  # Obtener la frecuencia de muestreo original

    # Resamplear a 16 kHz si es necesario
    if original_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=original_rate, new_freq=16000)
        waveform = resampler(waveform)

    batch["speech"] = waveform
    batch["sampling_rate"] = 16000  # Actualizar la frecuencia de muestreo
    return batch

# Procesar el dataset
processed_dataset = dataset.map(process_audio)


Map:   0%|          | 0/230467 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [2]:
import torchaudio
import torch

def resample_audio(batch):
    waveform, sample_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    batch["speech"] = resampler(waveform).squeeze(0)  # Mantén el audio como un tensor
    batch["sampling_rate"] = 16000  # Añadir la frecuencia de muestreo
    return batch

# Procesar los datos nuevamente
resampled_dataset = dataset.map(resample_audio)



In [ ]:
resampled_dataset.save_to_disk("./processed_common_voice")



Saving the dataset (0/179 shards):   0%|          | 0/230467 [00:00<?, ? examples/s]

In [8]:
from datasets import load_from_disk
import os
# Obtener la ruta actual
current_path = os.getcwd()
print(f"Ruta actual: {current_path}")

# Especificar el nombre del directorio para guardar/cargar el dataset
dataset_path = os.path.join(current_path, "processed_common_voice")

# Cargar el dataset tokenizado
resampled_dataset = load_from_disk(dataset_path)
print(resampled_dataset)

Ruta actual: c:\Proyectos\Film_recomendations


Loading dataset from disk:   0%|          | 0/179 [00:00<?, ?it/s]

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech', 'sampling_rate'],
    num_rows: 230467
})


In [9]:
# Validar un ejemplo
example = resampled_dataset[0]
print(f"Texto: {example['sentence']}")
print(f"Forma de audio: {example['speech'].shape}")


Texto: ¿ Qué tal a tres de cinco ?


AttributeError: 'list' object has no attribute 'shape'

In [7]:
from transformers import SpeechT5Processor

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

def tokenize_text(batch):
    inputs = processor(text=batch["sentence"], return_tensors="pt", padding=True, truncation=True)
    batch["input_ids"] = inputs.input_ids[0]
    return batch

tokenized_dataset = resampled_dataset.map(tokenize_text)


In [5]:
def filter_short(batch):
    return len(batch["input_ids"]) > 10 and batch["speech"].shape[0] > 1600  # Al menos 0.1 segundos de audio

filtered_dataset = tokenized_dataset.filter(filter_short)


Filter:   0%|          | 0/230467 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [10]:
tokenized_dataset.save_to_disk('./tokenized_dataset')

Saving the dataset (0/179 shards):   0%|          | 0/230467 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk
import os
# Obtener la ruta actual
current_path = os.getcwd()
print(f"Ruta actual: {current_path}")

# Especificar el nombre del directorio para guardar/cargar el dataset
dataset_path = os.path.join(current_path, "tokenized_dataset")

# Cargar el dataset tokenizado
tokenized_dataset = load_from_disk(dataset_path)
print(tokenized_dataset)

Ruta actual: c:\Proyectos\Film_recomendations


Loading dataset from disk:   0%|          | 0/179 [00:00<?, ?it/s]

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech', 'sampling_rate', 'input_ids'],
    num_rows: 230467
})


In [7]:
for example in tokenized_dataset:
    print(f"input_ids shape: {len(example['input_ids'])}")
    print(f"speech shape: {example['speech'].shape}")
    break


input_ids shape: 29


AttributeError: 'list' object has no attribute 'shape'

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
)


c:\Users\alexc\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, Trainer
from transformers import DataCollatorWithPadding

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

data_collator = DataCollatorWithPadding(tokenizer=processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,  # Agregar el collator
)

